In [1]:
%load_ext autoreload
%autoreload 2

import sys

path = "/app"
sys.path.insert(1, path)

from pathlib import Path
import pickle
from tqdm import tqdm
import numpy as np
from face_lib.datasets import IJBDataset, IJBATest, IJBCTemplates
from face_lib.evaluation.aggregation import aggregate_templates

In [1]:
d = {"a": 5, "b": 6}
str(d)

"{'a': 5, 'b': 6}"

In [2]:
cache_dir = "/app/cache"
uncertainty_strategy = "head"
dataset_path = "/app/data/big"
protocol_path = "/app/data/test1"

testset = IJBDataset(dataset_path)
image_paths = testset["abspath"].values

features_path = Path(cache_dir) / f"{uncertainty_strategy}_features.pickle"
uncertainty_path = Path(cache_dir) / f"{uncertainty_strategy}_uncertainty.pickle"

with open(features_path, "rb") as f:
    feature_dict = pickle.load(f)
with open(uncertainty_path, "rb") as f:
    uncertainty_dict = pickle.load(f)

In [3]:
tester = IJBCTemplates(image_paths, feature_dict, uncertainty_dict)
tester.init_proto(protocol_path)

normalize = True
aggregate_templates(tester.enroll_templates(), "PFE", normalize=normalize)
aggregate_templates(tester.verification_templates(), "first", normalize=normalize)



Cosine distance:
\begin{equation*}
    a^{ijl} = \sum_{k=1}^d\left[(z^{il}_k\cdot\hat{\mu}^j_k)\right]
\end{equation*}
Default PFE:
\begin{equation*}
    a^{ijl} = -\frac{1}{2}\sum_{k=1}^d\left[(z^{il}_k-\hat{\mu}^j_k)^2\frac{1}{{\hat{\sigma}^j_k}^2} +\log{\hat{\sigma}^j_k}^2\right]
\end{equation*}

\begin{equation*}
    a^{ijl} = -\frac{1}{2}\sum_{k=1}^d\left[\left(\frac{z^{il}_k}{\hat{\sigma}^j_k} -\frac{\hat{\mu}^j_k}{\hat{\sigma}^j_k}\right)^2 +\log{\hat{\sigma}^j_k}^2\right]
\end{equation*}

Cosine PFE:
\begin{equation*}
    a^{ijl} = \frac{1}{2}\sum_{k=1}^d\left[(z^{il}_k\cdot\hat{\mu}^j_k)\frac{1}{{\hat{\sigma}^j_k}^2} \cancel{-\log{\hat{\sigma}^j_k}^2}\right]
\end{equation*}



In [4]:
from multiprocessing import Pool


class Compute_scores:
    def __init__(self, verif_mus) -> None:
        self.verif_mus = verif_mus

    def __call__(self, data_piece):
        mu, sigma_sq = data_piece
        enroll_template_sigma_sqrt = np.sqrt(sigma_sq)
        log_sigma_sum = np.log(sigma_sq).sum()
        z_dist_to_mu = []
        z_dist_to_mu_over_sigma = []
        for mu_ver in self.verif_mus:
            z_dist_to_mu.append(np.linalg.norm(mu_ver - mu) ** 2)
            z_dist_to_mu_over_sigma.append(
                np.linalg.norm((mu_ver - mu) / enroll_template_sigma_sqrt) ** 2
            )
        return log_sigma_sum, z_dist_to_mu, z_dist_to_mu_over_sigma


enroll_data = [(t.mu, t.sigma_sq) for t in tester.enroll_templates()]
verification_mus = [t.mu for t in tester.verification_templates()]

In [5]:
compute_scores = Compute_scores(verification_mus)

with Pool(40) as pool_fc:
    data = []  # pool_fc.map(compute_scores, enroll_data)
    for result in tqdm(
        pool_fc.imap_unordered(compute_scores, enroll_data), total=len(enroll_data)
    ):
        data.append(result)

100%|██████████| 3497/3497 [20:41<00:00,  2.82it/s]  


In [42]:
z_dist_to_mu = []
log_sigma_sum = []
z_dist_to_mu_over_sigma = []

for t in data:
    log_sigma_sum.append(t[0])
    z_dist_to_mu.append(t[1])
    z_dist_to_mu_over_sigma.append(t[2])

In [43]:
sigma_sq = []
enroll_mu = []
for t in enroll_data:
    sigma_sq.append(t[1])
    enroll_mu.append(t[0])

z_dist_to_mu = np.array(z_dist_to_mu)
log_sigma_sum = np.array(log_sigma_sum)
z_dist_to_mu_over_sigma = np.array(z_dist_to_mu_over_sigma)
verification_mus = np.array(verification_mus)
enroll_mu = np.array(enroll_mu)
sigma_sq = np.array(sigma_sq)

In [44]:
z_dist_to_mu_min = np.min(z_dist_to_mu, axis=0)
np.mean(z_dist_to_mu_min), np.std(z_dist_to_mu_min)

(0.7787896389670563, 0.2901087908616003)

In [45]:
enroll_norm = np.linalg.norm(enroll_mu, axis=1)
np.mean(enroll_norm), np.std(enroll_norm)

(0.89221215, 0.08975799)

In [57]:
veverification_mus_norm = np.linalg.norm(verification_mus, axis=1)
np.mean(veverification_mus_norm), np.std(veverification_mus_norm)

(1.0, 3.642485e-08)

In [12]:
log_sigma_sum.mean(), log_sigma_sum.std()

(-3727.771, 192.98288)

In [13]:
z_dist_to_mu_over_sigma.mean(), z_dist_to_mu_over_sigma.std()

(2787.699437101349, 1022.7334336205469)

In [14]:
sigma_sq.mean(), sigma_sq.std()

(0.00074784114, 0.0003429491)

## Likelyhoods compare

In [57]:
z = []  # n x num_z_samples x 512
sigma = []  # K x 512
mu = []  # K x 512

# sample z's for each query image
for query_template in tqdm(tester.verification_templates()):
    z_samples = []
    z_samples.append(query_template.mu)
    z.append(z_samples)
z = np.array(z)
for enroll_template in tester.enroll_templates():
    sigma.append(enroll_template.sigma_sq)
    mu.append(enroll_template.mu)
sigma = np.array(sigma)
mu = np.array(mu)

100%|██████████| 19467/19467 [00:00<00:00, 411694.34it/s]


In [112]:
sigma = sigma / (np.min(sigma))

In [113]:
# norm_sigma
np.min(sigma)

1.0

In [114]:
sigma.mean()

4.948837

In [115]:
np.max(sigma)

29.736185

In [116]:
z_range = [22, 44, 566, 444, 19466]
a_1 = default_pfe_likelihood(mu, sigma, z[z_range]) * -2
a_2 = default_pfe_likelihood_slow(mu, sigma, z[z_range]) * -2

np.allclose(a_1, a_2)

100%|██████████| 512/512 [00:02<00:00, 203.95it/s]


True

In [117]:
a_1.mean()

777.0017

In [118]:
diff = np.abs(a_1 - a_2)[:, 0, :]

In [119]:
np.unravel_index(diff.argmax(), diff.shape)

(2, 853)

In [120]:
diff.max()

0.0002619028091430664